In [1]:
import ast
from structlog import get_logger
import pandas as pd
import numpy as np

In [2]:
datas = pd.DataFrame(np.random.randint(0, 10, size=(10, 2)), columns=[f'col_{n+1}' for n in range(2)])

In [3]:
logger = get_logger()

In [4]:
import operator

In [5]:
def eval_expression(node: ast.AST, datas:pd.DataFrame):
    log = logger.bind(func='eval expression')
    log.msg(node)
    return eval_node(node.body, datas=datas)

def eval_constant(node: ast.AST, datas:pd.DataFrame):
    log = logger.bind(func='eval constant')
    log.msg(node)
    return node.value

def eval_name(node: ast.AST, datas:pd.DataFrame):
    log = logger.bind(func='eval name')
    log.msg(node)
    if node.id in datas.columns:
        return datas[node.id]    

def eval_binop(node: ast.AST, datas:pd.DataFrame):
    log = logger.bind(func='eval binop')
    log.msg(node)
    left_value = eval_node(node.left, datas=datas)
    right_value = eval_node(node.right, datas=datas)
    apply = BINARY_OPERATIONS[type(node.op)]
    return apply(left_value, right_value)

def eval_unaryop(node: ast.AST, datas:pd.DataFrame):
    log = logger.bind(func='eval unaryop')
    log.msg(node)
    operand_value = eval_node(node.operand, vars)
    apply = UNARY_OPERATIONS[type(node.op)]
    return apply(operand_value)

In [6]:
EVALUATORS = {
        ast.Expression: eval_expression,
        ast.Constant: eval_constant,
        ast.Name: eval_name,
        ast.BinOp: eval_binop,
        ast.UnaryOp: eval_unaryop,
    }

BINARY_OPERATIONS = {
    ast.Add: operator.add,
    ast.Sub: operator.sub,
    ast.Mult: operator.mul,
    ast.Div: operator.truediv,
    ast.Pow: operator.pow
}

UNARY_OPERATIONS = {
    ast.USub: operator.neg,
}

In [7]:
def eval_node(node: ast.AST, datas:pd.DataFrame):
    log = logger.bind(func='eval node')
    log.msg(node)
    node_type = type(node)
    assert node_type in EVALUATORS.keys()
    
    func = EVALUATORS[node_type]
    return func(node, datas=datas)

In [8]:
rule1 = "(col_1 + 1)**2 - (2 * 3 / -4) + (col_2 * 4) + col_1 ** 2 - col_2 / 4 + col_1**(-0.4)"

In [9]:
(datas.col_1 + 1)**2 - (2*3 / -4) + (datas.col_2 * 4) + datas.col_1**2 -datas.col_2 / 4 + datas.col_1**(-0.4)

0     78.025306
1     15.257858
2    137.459157
3     30.257858
4     37.757858
5     69.324349
6    146.935275
7     45.894394
8           inf
9    182.915244
dtype: float64

In [10]:
node = ast.parse(rule1, '<string>', mode='eval')

In [11]:
eval_node(node, datas=datas)

2022-10-30 11:51.16 [info     ] <ast.Expression object at 0x0000019730FB8CD0> func=eval node
2022-10-30 11:51.16 [info     ] <ast.Expression object at 0x0000019730FB8CD0> func=eval expression
2022-10-30 11:51.16 [info     ] <ast.BinOp object at 0x0000019730FBBBB0> func=eval node
2022-10-30 11:51.16 [info     ] <ast.BinOp object at 0x0000019730FBBBB0> func=eval binop
2022-10-30 11:51.16 [info     ] <ast.BinOp object at 0x0000019730FBBC40> func=eval node
2022-10-30 11:51.16 [info     ] <ast.BinOp object at 0x0000019730FBBC40> func=eval binop
2022-10-30 11:51.16 [info     ] <ast.BinOp object at 0x0000019730FBBCD0> func=eval node
2022-10-30 11:51.16 [info     ] <ast.BinOp object at 0x0000019730FBBCD0> func=eval binop
2022-10-30 11:51.16 [info     ] <ast.BinOp object at 0x0000019730FBBD30> func=eval node
2022-10-30 11:51.16 [info     ] <ast.BinOp object at 0x0000019730FBBD30> func=eval binop
2022-10-30 11:51.16 [info     ] <ast.BinOp object at 0x0000019730FBBCA0> func=eval node
2022-10-30 1

0     78.025306
1     15.257858
2    137.459157
3     30.257858
4     37.757858
5     69.324349
6    146.935275
7     45.894394
8           inf
9    182.915244
dtype: float64

In [15]:
from pydantic import BaseModel
from enum import Enum, auto

In [16]:
class NodeType(Enum):
    Expression = auto()
    Constant = auto()
    Variable = auto()

In [17]:
class Node:
    def __init__(self, node_type, left, right):
        self.node_type = node_type
        self.left = left
        self.right = right

In [19]:
ast.dump(node)

"Expression(body=BinOp(left=BinOp(left=BinOp(left=BinOp(left=BinOp(left=BinOp(left=BinOp(left=Name(id='col_1', ctx=Load()), op=Add(), right=Constant(value=1)), op=Pow(), right=Constant(value=2)), op=Sub(), right=BinOp(left=BinOp(left=Constant(value=2), op=Mult(), right=Constant(value=3)), op=Div(), right=UnaryOp(op=USub(), operand=Constant(value=4)))), op=Add(), right=BinOp(left=Name(id='col_2', ctx=Load()), op=Mult(), right=Constant(value=4))), op=Add(), right=BinOp(left=Name(id='col_1', ctx=Load()), op=Pow(), right=Constant(value=2))), op=Sub(), right=BinOp(left=Name(id='col_2', ctx=Load()), op=Div(), right=Constant(value=4))), op=Add(), right=BinOp(left=Name(id='col_1', ctx=Load()), op=Pow(), right=UnaryOp(op=USub(), operand=Constant(value=0.4)))))"